# Using Cassandra and Spark
**Andrey Titov**, andrey.titov@bigdatateam.org

Big Data Instructor @ BigData Team, http://bigdatateam.org/

## На этом занятии
+ Зачем нужны NoSQL
+ Теорема CAP Брюера
+ Архитектура БД Cassandra
+ Модель данных
+ Чтение и фильтрация
+ Удаление данных
+ Запись и изменение данных
+ Работа с Cassandra в Spark

# Зачем нужны NoSQL

NoSQL - термин, описывающий класс БД, имеющих архитектурные отличия от классических реляционных БД.  
Основными факторами развития NoSQL БД считаются:
- скорость чтения/записи
- объем данных
- ACID не всегда нужен

## Пример
Вы проектируете платформу сбора данных с различных датчиков автомобилей всего мира:
- критична ли потеря одного события? (нет)
- что нам важнее - консистентность данных или доступность системы? (доступность)
- будут ли проблемы с сетью? (да)
- сколько событий в секунду мы будем обрабатывать? (неизвестно)

Для этой задачи нам потребуется БД, которая:
- является **гео распределенной**
- обеспечивает **доступность данных** при выходе из строя любого узла
- продолжает работу при **нарушении сетевой связанности** между любыми узлами
- умеет **горизонтально масштабироваться**

# Теорема CAP Брюера

В любой распределенной БД возможно гарантировать выполнение **только двух из трех** свойств:
- Consistency
- Availability
- Partitition tolerance

**Consistency**  
Результат любого запроса проявляется везде и сразу после того, как мы получили подтверждение от узла о его выполнении
  
**Availability**  
Любая доступная нода должна ответить на запрос

**Partition tolerance**  
Система продолжает работать в условиях нарушения сетевой связности

## И что это означает?

**CA системы**  
При возникновении проблем все ноды перестают обрабатывать запросы, но зато все консистентно :3

**CP системы**  
В случае проблем никто не гарантирует доступность данных

**AP системы**  
Система будет доступна даже после ядерного апокалипсиса, но некоторое время может возвращать не то, что вы ожидаете

# Архитектура БД Cassandra

Cassandra - AP система в теореме CAP. На практике это означает:
- высокая доступность данных
- нет транзакций (не совсем)
- можно строить гео-кластера
- слабая согласованность (eventual)
- линейная масштабируемость
- высокая пропускная способность (особенно на запись)

Cassandra имеет симметричную архитектуру. Каждый узел отвечает за хранение данных, обработку запросов и состояние кластера. 

Расположение данных определяется значением хеш функции от Partition key.

Высокая доступность данных обеспечивается за счет репликации.

![Cassandra Architecture](https://cassandra.apache.org/doc/latest/_images/ring.svg)
Источник: https://cassandra.apache.org/doc/latest/architecture/dynamo.html#dataset-partitioning-consistent-hashing

# Подготовка данных

In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import dict_factory
from tabulate import tabulate

cluster = Cluster(['brain-node1'])
session = cluster.connect()
session.row_factory = dict_factory

In [ ]:
def print_table(data):
    print(
        tabulate(
            data, 
            tablefmt="pretty", 
            headers="keys", 
            showindex="always", 
            numalign="right", 
            stralign="right")
    )

In [ ]:
create_keyspace = \
"""
CREATE  KEYSPACE IF NOT EXISTS test 
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 3}
""" 

session.execute(create_keyspace)

In [ ]:
create_table = \
"""
CREATE TABLE IF NOT EXISTS test.cars0 (
    brand text PRIMARY KEY,
    model text,
    engine text,
    drive_wheel text,
    turbo boolean,
    acceleration float
)
"""

session.execute(create_table)

In [ ]:
import random

brands = [
  "Abarth",
  "Alfa Romeo",
  "Aston Martin",
  "Audi",
  "Bentley",
  "BMW",
  "Bugatti",
  "Cadillac",
  "Chevrolet",
  "Chrysler",
  "Citroën",
  "Dacia",
  "Daewoo",
  "Daihatsu",
  "Dodge",
  "Donkervoort",
  "DS",
  "Ferrari",
  "Fiat",
  "Fisker",
  "Ford",
  "Honda",
  "Hummer",
  "Hyundai",
  "Infiniti",
  "Iveco",
  "Jaguar",
  "Jeep",
  "Kia",
  "KTM",
  "Lada",
  "Lamborghini",
  "Lancia",
  "Land Rover",
  "Landwind",
  "Lexus",
  "Lotus",
  "Maserati",
  "Maybach",
  "Mazda",
  "McLaren",
  "Mercedes-Benz",
  "MG",
  "Mini",
  "Mitsubishi",
  "Morgan",
  "Nissan",
  "Opel",
  "Peugeot",
  "Porsche",
  "Renault",
  "Rolls-Royce",
  "Rover",
  "Saab",
  "Seat",
  "Skoda",
  "Smart",
  "SsangYong",
  "Subaru",
  "Suzuki",
  "Tesla",
  "Toyota",
  "Volkswagen",
  "Volvo"
]

models = [
    "alpha",
    "beta",
    "gamma",
    "delta",
    "epsilon",
    "varepsilon",
    "zeta",
    "eta",
    "theta",
    "iota",
    "kappa",
    "lambda",
    "mu",
    "nu",
    "xi",
    "omicron",
    "pi",
    "rho",
    "sigma",
    "tau",
    "upsilon",
    "phi",
    "varphi",
    "chi",
    "psi",
    "omega"
]

engines = ["petrol", "diesel", "electric", "hybrid"]

wheel_drive = ["rear", "front", "all"]

turbo = [False, True]

acceleration = list(range(3, 20))

insert_cars = \
"""
INSERT INTO {keyspace}.{table_name} (brand, model, engine, drive_wheel, turbo, acceleration)
VALUES (%s, %s, %s, %s, %s, %s)
    
"""


def write_data(keyspace, table_name, num_rows):
    for i in range(0, num_rows):
        data = (
            random.choice(brands),
            random.choice(models),
            random.choice(engines),
            random.choice(wheel_drive),
            random.choice(turbo),
            float(random.choice(acceleration))
        )
        session.execute(insert_cars.format(keyspace=keyspace, table_name=table_name), data)
    print("Written {n} rows".format(n=num_rows))
    
def truncate_table(table_name):
    truncate_query = "TRUNCATE TABLE test.{table_name}"
    session.execute(truncate_query.format(table_name=table_name))

In [ ]:
write_data("test", "cars0", 1000)

In [ ]:
select_cars = \
"""
SELECT * FROM test.cars0
"""

rows = session.execute(select_cars)
print_table(rows)

# Модель данных

В Cassandra существует три основных типа колонок:
- Обычные колонки
 + опциональны
 + могут быть иметь любой тип из поддерживаемых
 + являются nullable
 + не могут выступать в качестве условия фильтрации
 + можно добавлять новые и удалять колонки из таблицы
- Проиндексированные обычные колонки
 + могут выступать в качестве условия фильтрации
- Partition key
 + обязателен
 + порядок (если используется несколько partition key)
 + одна или несколько колонок
 + определяет физическое расположение данных на кластере
 + может выступать в качестве условия фильтрации с предикатами: =, IN
 + не все типы данных поддерживаются
- Clustering key
 + опционален
 + одна или несколько колонок
 + порядок (если используется несколько clustering key) имеет значение
 + определяет расположение данны внутри партиции
 + может выступать в качестве условия фильтрации с предикатами =, IN при соблюдении порядка следования*
 + последний clustering key в запросе может выступать в качестве условия фильтрации с предикатами <, >, !=, =, IN
 
Важно:
- строки внутри партиций отсортированы по clustering key
- composite key = partition key + clustering key
- composite key является уникальным ключом колонки
- в одной партиции не может быть более 2kkk строк

**Выводы:**
- **в реляционных БД модель данных определяется, исходя из структуры данных**
- **в Cassandra модель данных определяется, исходя из запросов к данным**
- **В Cassandra данные обычно хранят в денормализованном виде**

# Чтение и фильтрация
Partition key может выступать условием с предикатами =, IN:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars0 WHERE brand = 'Audi'"

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars0 WHERE brand IN ('Audi', 'BMW')"

Фильтровать данные по обычной колонке нельзя:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars0 WHERE drive_wheel = 'front'"

Но если очень хочется, то можно:

In [ ]:
# В большинстве случаев, использование ALLOW FILTERING - это антипаттерн
!cqlsh brain-node1 -e "SELECT * FROM test.cars0 WHERE drive_wheel = 'front' ALLOW FILTERING"

Вернемся к DDL таблицы cars0:
```
CREATE TABLE IF NOT EXISTS test.cars0 (
    brand text PRIMARY KEY,
    model text,
    engine text,
    drive_wheel text,
    turbo boolean,
    acceleration float
)
```

В данной таблице только одна колонка является единственным partition key - `brand`. Поэтому, сколько бы мы данных не записали, количество строк в таблице будет ограничено размером массива `brands`, остальные строки будут перезаписываться.

In [ ]:
len(brands)

При необходимости мы можем очистить таблицу cars0:

In [ ]:
truncate_table("cars0")

Создадим и наполним новую таблицу cars1, в которой ключами будут две колонки `brand` и `model`:

In [ ]:
create_table = \
"""
CREATE TABLE IF NOT EXISTS test.cars1 (
    brand text,
    model text,
    engine text,
    drive_wheel text,
    turbo boolean,
    acceleration float, PRIMARY KEY (brand, model)
)
"""

session.execute(create_table)
truncate_table("cars1")
write_data("test", "cars1", 1000)

Прочитаем все данные из этой таблицы и убедимся, что их больше `len(brands)`

In [ ]:
select_cars = \
"""
SELECT * FROM test.cars1;
"""

rows = session.execute(select_cars)
print("Table contains {n} rows and brands has length of {m}".format(n=len(list(rows)), m=len(brands)))

Выведем первые 5 строк на экран. Красным цветом выделен partition key, голубым - clustering key:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars1 LIMIT 5"

Фильтровать данные запросы здесь можно:
- по полю `brand`
- по полю `brand` и полю `model`

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars1 WHERE brand = 'Chrysler' LIMIT 5"

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars1 WHERE brand = 'Chrysler' AND model = 'alpha'"

Фильтровать данные только по полю `model` нельзья:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars1 WHERE model = 'alpha'"

Создадим еще одну таблицу, используя колонки `brand`, `model`, `engine`, `acceleration` в качестве ключей:

In [ ]:
create_table = \
"""
CREATE TABLE IF NOT EXISTS test.cars2 (
    brand text,
    model text,
    engine text,
    drive_wheel text,
    turbo boolean,
    acceleration float, PRIMARY KEY ((brand, model), engine, acceleration)
)
"""

session.execute(create_table)
truncate_table("cars2")
write_data("test", "cars2", 10000)

In [ ]:
select_cars = \
"""
SELECT * FROM test.cars2;
"""

rows = session.execute(select_cars)
print("Table contains {n} rows".format(n=len(list(rows))))

Изучим структуру ключей в таблице:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 LIMIT 5"

Возможные варианты фильтрации:
- по колонкам `brand`, `model`
- по колонкам `brand`, `model`, `engine`,
- по колонка `brand`, `model`, `engine`, `acceleration`

Важно:
- фильтрация по колонкам `brand` и `model` возможна только используя условия = и IN
- фильтровать <, >, != можно только по последнему кластерному ключу в запросе

Таким образом, работать будут следующие запросы:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Volvo' and model = 'phi'"

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Volvo' and model = 'phi' and engine = 'electric'"

In [ ]:
!cqlsh brain-node1 -e \
    "SELECT * FROM test.cars2 WHERE brand = 'Volvo' and model = 'phi' and engine = 'electric' \
    AND acceleration > 10"

Запросы ниже работать не будут:

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE model = 'phi'"

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE engine = 'electric'"

In [ ]:
!cqlsh brain-node1 -e \
    "SELECT * FROM test.cars2 WHERE brand = 'Volvo' and model = 'phi' \
    AND acceleration > 10"

Выводы:
- структура таблицы в БД зависит от запросов к ней
- нельзя пропускать (слева направо) ключи при фильтрации, но можно опустить последние n кластерных ключей
- нельзя фильтровать по одному из partition keys
- нельзя фильтровать по обычным колонкам (если она не проиндексирована)
- фильтровать c использованием <, >, != можно только по последнему кластерному ключу в запросе

# Удаление данных
Для изучения запросов на удаление данных будем использовать таблицу `cars2`
Для начала, добавим в нее данных:

In [ ]:
write_data("test", "cars0", 10000)

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

In [ ]:
!cqlsh brain-node1 -e "DELETE FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi' \
    AND engine = 'diesel' AND acceleration > 1"
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

In [ ]:
!cqlsh brain-node1 -e "DELETE FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi' \
    AND engine = 'petrol'"
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

In [ ]:
!cqlsh brain-node1 -e "DELETE FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

Выводы:
- удалять можно одну строку, указав весь composite key
- удалять можно группу строк, указав все partition key и часть clustering key
- удалять можно партицию целиком

# Запись и изменение данных
Для изучения запросов на удаление данных будем использовать таблицу `cars2`
Для начала, добавим в нее данных:

In [ ]:
write_data("test", "cars2", 10000)

In [ ]:
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

In [ ]:
!cqlsh brain-node1 -e "INSERT INTO test.cars2 (brand, model, engine, acceleration, drive_wheel, turbo) \
    VALUES ('Audi', 'pi', 'electric', -1, 'all', false)"
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

In [ ]:
!cqlsh brain-node1 -e "UPDATE test.cars2 SET turbo = true \
    WHERE brand = 'Audi' AND model = 'pi' AND engine = 'electric' AND acceleration = -1"
!cqlsh brain-node1 -e "SELECT * FROM test.cars2 WHERE brand = 'Audi' AND model = 'pi'"

Выводы:
- под капотом INSERT и UPDATE являются единой операцией, которая называется UPSERT
- при использовании INSERT и UPDATE единственным требованием является указание всего composite key
- используя стандартный SELECT, INSERT и UPDATE, нельзя обеспечить атомарное изменение поля строки (см. LWT https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useInsertLWT.html)

# Spark cassandra connector

https://github.com/datastax/spark-cassandra-connector

Для работы с Cassandra в Spark необходимо добавить:
```
--packages com.datastax.spark:spark-cassandra-connector_2.11:2.4.3 \
--conf spark.cassandra.connection.host=brain-node1 \
```

In [ ]:
from pyspark.sql.functions import *

df = spark.read \
            .format("org.apache.spark.sql.cassandra") \
            .options(table="cars0", keyspace="test") \
            .load()
    
df.printSchema()

df.show()

Применяя фильтр, следует помнить о структуре ключей. Если фильтр составлен правильном, то сработает predicate pushdown:

In [ ]:
filtered_p = df.filter(col("brand") == "Audi")
filtered_p.explain(True)
filtered_p.show()

Predicate pushdown сработает и в этом случае, но под капотом будет использован ALLOW FILTERING, т.к. фильтрация осуществляется не по ключу:

In [ ]:
filtered_p = df.filter(col("engine") == "petrol")
filtered_p.explain(True)
filtered_p.show()

Однако, если сделать более сложный фильтр, то predicate pyshdown не произойдет. В этом случае spark прочитает таблицу ЦЕЛИКОМ. Это следует помнить при работе с большими таблицами:

In [ ]:
filtered_p = df.filter(length(col("brand")) > 4)
filtered_p.explain(True)
filtered_p.show()

Запись осуществляется по аналогии с другими форматами:

In [ ]:
filtered_p \
    .write \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="cars1", keyspace="test").mode("append").save()

## Full List of Predicate Pushdown Restrictions

1. Only push down no-partition key column predicates with =, >, <, >=, <= predicate
2. Only push down primary key column predicates with = or IN predicate.
3. If there are regular columns in the pushdown predicates, they should have
   at least one EQ expression on an indexed column and no IN predicates.
4. All partition column predicates must be included in the predicates to be pushed down,
   any part of the partition key can be an EQ or IN predicate. For each partition column,
   only one predicate is allowed.
5. For cluster column predicates, only last predicate can be RANGE predicate
   and preceding column predicates must be EQ or IN predicates.
   If there is only one cluster column predicate, the predicates could be EQ or IN or RANGE predicate.
6. There is no pushdown predicates if there is any OR condition or NOT IN condition.
7. We're not allowed to push down multiple predicates for the same column if any of them
   is equality or IN predicate.
   
https://github.com/datastax/spark-cassandra-connector/blob/master/doc/14_data_frames.md#full-list-of-predicate-pushdown-restrictions

Выводы:
- используя cassandra в spark, следует помнить о структуре композитного ключа и особенностях составления запросов к БД
- конфигурация БД указывается в параметрах `spark-submit` при запуске приложения
- для работы с Cassandra необходимо добавить зависимость с https://mvnrepository.com

# Workshop

Эффективность работы Cassandra в проекте напрямую зависит от структуры таблиц. Перед тем как создавать таблицы, вам необходимо понять, какие запросы будут делать пользователи. На этом семинаре вам нужно подготовить данные, загрузить их в БД и ответить на вопросы ниже

Датасет: https://datahub.io/core/airport-codes#resource-airport-codes

In [ ]:
from pyspark.sql.functions import *
df = spark.read.csv(path="/tmp/airport-codes.csv", header=True)
df.printSchema()
print("#" * 20)
print("There are {c} airports in the dataset".format(c=df.count()))
print("#" * 20)
df.show(1, 200, True)

## Задача 1

В Сassandra все таблицы сгруппированы по KEYSPACE'ам. На уровне KEYSPACE'а устанавливаются важный параметр - фактор репликации данных в таблице. В случае геораспределенных инсталляций фактор репликации задается для каждого датацентра отдельно.

Создайте KEYSPACE с именем **вашего пользователя** с параметром `REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 3}`

In [ ]:
df.filter(col("iso_country") == "RU").groupBy("iso_region").count().show(50, False)

Теперь вам нужно создать таблицу и загрузить в нее данные. Тут есть несколько моментов:
- данные необходимо почистить и предобработать:
 + разбить coordinates на 2 колонки и преобразовать во float: первое значение это longitude, второе - lattitude
 + структура ключей будет такая: `((iso_country, iso_region), ident)` - кортеж iso_country и iso_region это partition key, а ident - clustrering key. Перед записью данных в БД нам нужно убедиться, что среди ключей нет null и что кортеж (iso_country, iso_region, ident) не имеет дубликатов в датасете, иначе мы не сможем использовать его в качестве ключа
- готовить и записывать данные удобнее с помощью Spark. При записи вам нужно обеспечить соответствие имен и типов полей DF полям таблицы

Создайте таблицу `airports_plain` в вашем keyspace

Выполните предобработку данных и запишите их в таблицу

## Задача 2

Прочитайте таблицу с помощью `spark`. Выведите первые 20 строк на экран

Повторите операцию, используя `cqlsh`

Повторите операцию, используя библиотеку `cassandra`

Выведите на экран 20 аэропортов, расположенных в `iso_country = RU` и `iso_region = RU-MOW`. Выполните эту операцию с помощью `spark`, `cassandra`

Выведите на экран 20 аэропортов, у которых `elevation_ft > 100`. Выполните эту операцию с помощью `spark`, `cassandra`

Выведите на экран 20 аэропортов, у которых значение `name` начинается с буквы `T`. Выполните эту операцию с помощью `spark`, `cassandra`

На основании выполненных экспериментов вы должны сделать следующие выводы:
- Пока запрос "укладывается" в структуру таблицы,  Cassandra будет отдавать данные очень быстро при использовании обычной библиотеки (не spark). Даже если речь будет идти о десятках тысячах запросов в секунду, БД справиться с этой задачей.
- Если запрос не "укладывается" в структуру, то у вас есть вариант использовать `ALLOW FILTERING`. Но здесь у вас будет деградация скорости выполнения запросов и избыточная нагрузка на базу.
- Если запрос не укладывается в ограничения синтаксиса CQL, то вашим единственным вариантом является прочитать большой объем данных из таблицы и затем обработать их. Именно это и делает за вас `spark` в третьем эксперименте. Однако, запустить большое количество одновременных запросов к базе с помощью `spark` не получится - база не сможет отдавать данные с такой скоростью. В этом от части и состоит прелесть Cassandra - при должном проектировании вы можете совмещать разные типы нагрузок и одновременно использовать ее для высоконагруженного бекенд сервиса и выполнять OLAP анализ данных на одних и тех же данных. В случае с HDFS такое возможным не представляется

## Задача 3

Спроектируйте таблицу (таблицы) и наполните их данными так, чтобы максимально быстро отвечать на запрос:
"Вывести список аэропортов, расположенных на расстоянии не более `max_longitude_dist` по долготе и `max_lattitude_dist` по широте". 

Напишите функцию, которая позволяет ответить на данный запрос. Данная функция принимает текущие координаты объекта (например, у нас летит самолет и ему надо срочно где-то выполнить посадку) и максимальное расстояние по долготе и широте. Функция должна возвращать список словарей с данными об аэропортах.

P.S. Для решения это задачи можете использовать как библиотеку `cassandra`, так и `spark`, но данная функция должна работать макисмально быстро.

P.P.S. Не всегда нужно хранить данные в одной таблице. Помните, в Cassandra таблицы делаются под запросы и денормализация данных - это нормально

In [ ]:
def get_nearest_airports(current_longitude, current_lattitude, max_longitude_dist, max_lattitude_dist):
    pass